In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, MultiHeadAttention, LayerNormalization, Dropout, Add, GlobalAveragePooling1D, Reshape
from tensorflow.keras.models import Model

In [4]:
# Load and preprocess the new dataset
path = "Dataset.csv"
data = pd.read_csv(path, parse_dates=['Datetime'])
data.set_index('Datetime', inplace=True)

# Handle missing values
data.ffill(inplace=True)

# Normalize features
scalers = {}
features = ['Temperature', 'Relative Humidity', 'Wind Speed', 'Precipitation', 'Is_Weekend_Holiday', 'AEP_MW']
for feature in features:
    scaler = MinMaxScaler()
    data[feature] = scaler.fit_transform(data[[feature]])
    scalers[feature] = scaler

# Prepare sequences for training
n_timesteps = 24  # Fixed size of previous 24 hours of context data
n_future = 48  # Maximum number of future hours to predict

X, Y = [], []
for i in range(len(data) - n_timesteps - n_future):
    X.append(data.iloc[i:i + n_timesteps][['Temperature', 'Relative Humidity', 'Wind Speed', 'Precipitation', 'Is_Weekend_Holiday']].values)
    Y.append(data.iloc[i + n_timesteps:i + n_timesteps + n_future]['AEP_MW'].values.reshape(-1, 1))  # Predict up to 48 hours

X, Y = np.array(X), np.array(Y)

# Split into training and testing
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
Y_train, Y_test = Y[:split], Y[split:]

In [2]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Multi-Head Self Attention
    attention = MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)
    x = attention(inputs, inputs)
    x = Dropout(dropout)(x)
    res = Add()([x, inputs])
    x = LayerNormalization(epsilon=1e-6)(res)

    # Feed Forward Part
    ff = Dense(ff_dim, activation="relu")(x)
    ff = Dropout(dropout)(ff)
    ff = Dense(inputs.shape[-1])(ff)
    x = Add()([x, ff])
    x = LayerNormalization(epsilon=1e-6)(x)
    return x

def build_flexible_transformer_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0, mlp_dropout=0):
    inputs = Input(shape=input_shape)
    x = inputs

    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = GlobalAveragePooling1D()(x)
    x = Dense(mlp_units, activation="relu")(x)
    x = Dropout(mlp_dropout)(x)

    # Output layer for maximum prediction length (48 hours)
    x = Dense(n_future)(x)
    outputs = Reshape((n_future, 1))(x)  # Reshape for maximum output length

    return Model(inputs, outputs)

In [3]:
# Build and compile the model once
input_shape = (n_timesteps, X_train.shape[2])  # Input shape is fixed to 24 hours of historical data

model = build_flexible_transformer_model(
    input_shape=input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=512,
    num_transformer_blocks=6,
    mlp_units=256,
    dropout=0.1,
    mlp_dropout=0.1
)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss="mse", metrics=["mae"])

# Train the model
history = model.fit(
    X_train, Y_train,
    validation_data=(X_test, Y_test),
    epochs=20,  # Increase epochs as needed
    batch_size=32,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
    ]
)

# Save the trained model
model.save('flexible_electricity_forecast_model.keras')

Epoch 1/20
3030/3030 ━━━━━━━━━━━━━━━━━━━━ 97s 18ms/step - loss: 0.0465 - mae: 0.1643 - val_loss: 0.0258 - val_mae: 0.1339 - learning_rate: 1.0000e-04
Epoch 2/20
3030/3030 ━━━━━━━━━━━━━━━━━━━━ 51s 17ms/step - loss: 0.0258 - mae: 0.1291 - val_loss: 0.0256 - val_mae: 0.1335 - learning_rate: 1.0000e-04
Epoch 3/20
3030/3030 ━━━━━━━━━━━━━━━━━━━━ 82s 17ms/step - loss: 0.0253 - mae: 0.1279 - val_loss: 0.0258 - val_mae: 0.1341 - learning_rate: 1.0000e-04
Epoch 4/20
3030/3030 ━━━━━━━━━━━━━━━━━━━━ 51s 17ms/step - loss: 0.0249 - mae: 0.1268 - val_loss: 0.0254 - val_mae: 0.1331 - learning_rate: 1.0000e-04
Epoch 5/20
3030/3030 ━━━━━━━━━━━━━━━━━━━━ 69s 12ms/step - loss: 0.0249 - mae: 0.1269 - val_loss: 0.0257 - val_mae: 0.1340 - learning_rate: 1.0000e-04
Epoch 6/20
3030/3030 ━━━━━━━━━━━━━━━━━━━━ 41s 12ms/step - loss: 0.0248 - mae: 0.1266 - val_loss: 0.0259 - val_mae: 0.1348 - learning_rate: 1.0000e-04
Epoch 7/20
3030/3030 ━━━━━━━━━━━━━━━━━━━━ 44s 13ms/step - loss: 0.0246 - mae: 0.1261 - val_loss: 0.0

In [8]:
def prepare_input_for_prediction(historical_data):
    """
    Prepares the input data for prediction, using only historical data.

    Args:
    - historical_data (pd.DataFrame): DataFrame with the historical data.

    Returns:
    - np.array: Prepared input data for the model.
    """
    historical_features = ['Temperature', 'Relative Humidity', 'Wind Speed', 'Precipitation', 'Is_Weekend_Holiday']
    # Ensure historical data has the correct features and is normalized
    historical_input = historical_data[historical_features].values
    # Normalize historical input data
    for feature in historical_features:
        historical_input[:, historical_features.index(feature)] = scalers[feature].transform(historical_input[:, historical_features.index(feature)].reshape(-1, 1)).reshape(-1)
    return historical_input.reshape(1, 24, len(historical_features))  # Reshape for model input

def predict_next_n_hours(model, historical_data, forecast_data, n_hours):
    # Prepare historical input
    historical_input = prepare_input_for_prediction(historical_data)

    # Prepare forecast input
    forecast_input = forecast_data.values.reshape(1, n_hours, 5)

    # Combine the most recent historical data with forecast data
    combined_input = np.concatenate([historical_input[:, -24+n_hours:, :], forecast_input], axis=1)

    # Ensure we have exactly 24 hours of input data
    if combined_input.shape[1] > 24:
        combined_input = combined_input[:, -24:, :]

    # Make prediction
    predictions = model.predict(combined_input)
    return scalers['AEP_MW'].inverse_transform(predictions[0, :n_hours].reshape(-1, 1))

In [3]:
def get_manual_input(hours, is_forecast=False):
    manual_data = []
    data_type = "forecast" if is_forecast else "historical"
    print(f"Please enter the {data_type} data for the {'next' if is_forecast else 'previous'} {hours} hours.")
    for i in range(hours):
        print(f"Hour {i+1}:")
        temperature = float(input("  Temperature: "))
        humidity = float(input("  Relative Humidity: "))
        wind_speed = float(input("  Wind Speed: "))
        precipitation = float(input("  Precipitation: "))
        is_weekend_holiday = int(input("  Is Weekend/Holiday (1 for Yes, 0 for No): "))
        manual_data.append([temperature, humidity, wind_speed, precipitation, is_weekend_holiday])

    columns = ['Temperature', 'Relative Humidity', 'Wind Speed', 'Precipitation', 'Is_Weekend_Holiday']
    df_manual = pd.DataFrame(manual_data, columns=columns)

    for feature in columns:
        df_manual[feature] = scalers[feature].transform(df_manual[[feature]])

    return df_manual

In [ ]:
# Load the trained model
model = tf.keras.models.load_model('flexible_electricity_forecast_model.keras')

# Ask user for the prediction horizon
n_hours = int(input("Enter the number of hours for prediction (3, 6, 12, 24, 48): "))
date = input("Enter the date for prediction (YYYY-MM-DD HH:MM:SS): ")

# Determine if historical data is available
historical_end = pd.to_datetime(date) - pd.Timedelta(hours=1)
historical_start = historical_end - pd.Timedelta(hours=23)

if historical_start not in data.index or historical_end not in data.index:
    print("Historical data not available in dataset. Please enter the previous 24 hours of data manually.")
    historical_data = get_manual_input(24)
else:
    historical_data = data.loc[historical_start:historical_end]
    print("Using historical data from the dataset.")

# Ensure we have 24 hours of historical data
if len(historical_data) < 24:
    print("Warning: Not enough historical data. Please provide the missing data manually.")
    missing_hours = 24 - len(historical_data)
    manual_historical = get_manual_input(missing_hours)
    historical_data = pd.concat([manual_historical, historical_data])

# Get forecast data for the next n_hours
print(f"\nNow, please enter the weather forecast data for the next {n_hours} hours:")
forecast_data = get_manual_input(n_hours, is_forecast=True)

# Predict the next N hours
predicted_values = predict_next_n_hours(model, historical_data, forecast_data, n_hours)

# Print the results
for i, value in enumerate(predicted_values):
    print(f"Hour {i+1}: {value[0]:.2f} MW")